<a href="https://colab.research.google.com/github/ekta2306/Online-Sexism-Detection/blob/main/HateXplain_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install ekphrasis

In [ ]:
pip install plotly==4.5.4

In [ ]:
!pip install transformers==4.2.1

In [ ]:
!pip install ekphrasis

In [ ]:
!pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [ ]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

In [ ]:
def print_text(texts,i,j):
    for u in range(i,j):
        print(texts[u])
        print()

In [ ]:
df = pd.read_csv('/content/train_new.csv',delimiter=',',encoding='utf-8')  # train set
print(list(df.columns.values)) #file header

In [ ]:
text_array = df["tweet"]
labels = df["class"]
original = text_array

In [ ]:
from collections import Counter

In [ ]:
df_validate = pd.read_csv('/content/val.csv',delimiter=',',encoding='utf-8') # validation set

In [ ]:
df_test = pd.read_csv('/content/test_new.csv',delimiter=',',encoding='utf-8') # test set

In [ ]:
#check if need to change
text_array_val = df_validate["tweet"]
labels_val = df_validate["class"]

In [ ]:
len(labels_val)

In [ ]:
test_set_text = df_test["tweet"]
labels_test_set = df_test["class"]

In [ ]:
import re

def remove_website(text):
    pattern = re.compile(r'https?://\S+|www\.\S+|\b\w+\.(com|co|net)\b', re.IGNORECASE)
    return " ".join(["" if pattern.search(word) else word for word in text.split()])

# Clean training set
text_array = text_array.apply(remove_website)
print_text(text_array, 0, 10)

print("**************************************************************************")

# Clean validation set
text_array_val = text_array_val.apply(remove_website)
print_text(text_array_val, 0, 10)

print("**************************************************************************")

# Clean test set
test_set_text = test_set_text.apply(remove_website)
print_text(test_set_text, 0, 10)


In [ ]:
# Functions for chat word conversion
f = open("/content/slang.txt", "r")
chat_words_str = f.read()
chat_words_map_dict = {}
chat_words_list = []

for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [ ]:
# Chat word conversion
# Training set
text_array = text_array.apply(lambda text: chat_words_conversion(text))
print_text(text_array,0,10)
print_text(original,0,10)

print("********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: chat_words_conversion(text))
print_text(text_array_val,0,10)

print("********************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: chat_words_conversion(text))
print_text(test_set_text,0,10)

In [ ]:
#check code in other copy
import re

# Define a dictionary of emoticons and their corresponding meanings
EMOTICONS = {
    ':)': 'smile',
    ':-)': 'smile',
    ':(': 'sad',
    ':-(': 'sad',
    ':D': 'laugh',
    '<3': 'love'
}

def convert_emoticons(text):
    # Iterate over each emoticon in the dictionary
    for emot, meaning in EMOTICONS.items():
        # Replace the emoticon with its corresponding meaning in the text
        text = re.sub(r'(?i)' + re.escape(emot), meaning, text)
    return text

# Test the emoticon conversion function
text = "Hello :-) :-("
text = convert_emoticons(text)
print(text)


In [ ]:
# Emoticon conversion
# Training set
text_array = text_array.apply(lambda text: convert_emoticons(text))
print_text(text_array,0,10)

print("**********************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emoticons(text))
print_text(text_array_val,0,10)

print("**********************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: convert_emoticons(text))
print_text(test_set_text,0,10)

In [ ]:
# FUnction for removal of emoji
import emoji

def convert_emojis(text):
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub("_|-"," ",text)
    return text

# Training set
text_array = text_array.apply(lambda text: convert_emojis(text))
print_text(text_array,0,10)

print("**************************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: convert_emojis(text))
print_text(text_array_val,0,10)

print("**************************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: convert_emojis(text))
print_text(test_set_text,0,10)

In [ ]:
# Ekphrasis pipe for text pre-processing
def ekphrasis_pipe(sentence):
    cleaned_sentence = " ".join(text_processor.pre_process_doc(sentence))
    return cleaned_sentence

# Training set
text_array = text_array.apply(lambda text: ekphrasis_pipe(text))
print("Training set completed.......")
#Validation set
text_array_val = text_array_val.apply(lambda text: ekphrasis_pipe(text))
print("Validation set completed.......")
#Test set
test_set_text = test_set_text.apply(lambda text: ekphrasis_pipe(text))
print("Test set completed.......")

In [ ]:
# Removing unnecessary punctuations
PUNCT_TO_REMOVE = "\"$%&'()+,-./;=[\]^_`{|}~"
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

# Training set
text_array = text_array.apply(lambda text: remove_punctuation(text))
print_text(text_array,0,10)

print("********************************************************************")

# Validation set
text_array_val = text_array_val.apply(lambda text: remove_punctuation(text))
print_text(text_array_val,0,10)

print("********************************************************************")

# Test set
test_set_text = test_set_text.apply(lambda text: remove_punctuation(text))
print_text(test_set_text,0,10)

In [ ]:
# Count of each label in dataset
from collections import Counter

# Printing training set counts for analysis
print("Elements: ",set(labels))
print("Length: ",len(labels))
print(Counter(labels))

print("**************************************************************************")

# Printing validation set counts for analysis
print("Elements: ",set(labels_val))
print("Length: ",len(labels_val))
print(Counter(labels_val))

print("**************************************************************************")

# Printing Test set counts for analysis
print("Elements: ",set(labels_test_set))
print("Length: ",len(labels_test_set))
print(Counter(labels_test_set))

In [ ]:
Y = []
Y_vali = []
Y_test_set = []

# Training set
for i in range(0, len(labels)):
    if labels[i] == 0:
        Y.append(0)
    elif labels[i] == 1:
        Y.append(1)
    elif labels[i] == 2:
        Y.append(2)

# Validation set
for i in range(0, len(labels_val)):
    if labels_val[i] == 0:
        Y_vali.append(0)
    elif labels_val[i] == 1:
        Y_vali.append(1)
    elif labels_val[i] == 2:
        Y_vali.append(2)

# Test set
for i in range(0, len(labels_test_set)):
    if labels_test_set[i] == 0:
        Y_test_set.append(0)
    elif labels_test_set[i] == 1:
        Y_test_set.append(1)
    elif labels_test_set[i] == 2:
        Y_test_set.append(2)


In [ ]:
len(labels)

In [ ]:
len(Y)

In [ ]:
len(labels_val)

In [ ]:
len(Y_vali)

In [ ]:
# Verifying train set
X = np.asarray(list(text_array))
Y = np.asarray(list(Y))
print(type(X))
print(type(Y))
print(np.shape(X),np.shape(Y))

# Verifying validation set
X_val = np.asarray(list(text_array_val))
Y_vali = np.asarray(list(Y_vali))
print(type(X_val))
print(type(Y_vali))
print(np.shape(X_val),np.shape(Y_vali))

# Verifying test set
X_test_set = np.asarray(list(test_set_text))
Y_test_set = np.asarray(list(Y_test_set))
print(type(X_test_set))
print(type(Y_test_set))
print(np.shape(X_test_set),np.shape(Y_test_set))

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
# Converting to one hot vectors
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)] #u[Y] helps to index each element of Y index at u. U here is a class array
    return Y

In [ ]:
Y_oh_train = convert_to_one_hot(np.array(Y), C = 3)
Y_oh_val = convert_to_one_hot(np.array(Y_vali), C = 3)
Y_oh_test_set = convert_to_one_hot(np.array(Y_test_set), C = 3)

In [ ]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter

In [ ]:
!pip install transformers
from transformers import RobertaTokenizerFast, TFRobertaModel, TFBertModel, BertTokenizerFast, ElectraTokenizerFast, TFElectraModel, AlbertTokenizerFast, TFAlbertModel, XLNetTokenizerFast, TFXLNetModel, MPNetTokenizerFast, TFMPNetModel
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import RepeatVector, Concatenate, Dense, Activation, Dot, BatchNormalization, Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
print(tf.__version__)

In [ ]:
import tensorflow as tf

# Check if a GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

# List available GPU devices
print("All devices: ", tf.config.list_logical_devices('GPU'))


In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

In [ ]:
X = list(X)
X_val = list(X_val)
X_test_set = list(X_test_set)

In [ ]:
len(X_val)

In [ ]:
len(Y_vali)

In [ ]:
model_train_x, model_val_x, Y_train, Y_val = train_test_split(X, Y, test_size=0.05, random_state=44)

In [ ]:
train_encodings = tokenizer(model_train_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
val_encodings = tokenizer(model_val_x, max_length=100, truncation=True, padding="max_length", return_tensors='tf')
test_encodings = tokenizer(X_val, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
cluster_encodings = tokenizer(X, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from transformers import TFRobertaModel
from tensorflow.keras.optimizers import Adam

def Offense_classifier(input_shape, dropout_rate=0.3, dense_units=256):
    roberta_model = TFRobertaModel.from_pretrained('roberta-base')

    input_ids = layers.Input(shape=input_shape, dtype=tf.int32, name="input_ids")
    attention_mask = layers.Input(shape=input_shape, dtype=tf.int32, name="attention_mask")

    def roberta_call(inputs):
        input_ids, attention_mask = inputs
        roberta_output = roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        return roberta_output.pooler_output

    roberta_embeddings = layers.Lambda(roberta_call, output_shape=(768,))([input_ids, attention_mask])

    x = layers.BatchNormalization()(roberta_embeddings)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='elu')(x)
    x = layers.Dense(32, activation='elu')(x)
    x = layers.Dense(3, activation='elu')(x)
    x = layers.Dense(32, activation='elu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(128, activation='elu')(x)

    output = layers.Dense(3, activation='softmax')(x)  # Updated for 3-class classification

    return models.Model(inputs=[input_ids, attention_mask], outputs=output)

# Use MirroredStrategy for distributed training (optional)
with tf.distribute.MirroredStrategy().scope():
    model = Offense_classifier((100,))  # Sequence length of 100 tokens

    optimizer = Adam(learning_rate=3e-5)
    loss_fun = tf.keras.losses.SparseCategoricalCrossentropy()  # Suitable for class indices 0/1/2
    metric = ['accuracy']

    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metric)

model.summary()


In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
from tensorflow import keras
import numpy as np
from sklearn.metrics import classification_report

class EvaluationMetric(keras.callbacks.Callback):

    def __init__(self, trial_encodings, trial_masks, Y_vali):
        super(EvaluationMetric, self).__init__()
        self.trial_encodings = trial_encodings
        self.trial_masks = trial_masks
        self.Y_vali = Y_vali

    def on_epoch_begin(self, epoch, logs=None):
        print("\nTraining...")

    def on_epoch_end(self, epoch, logs=None):
        print("\nEvaluating...")
        trial_prediction = self.model.predict([self.trial_encodings, self.trial_masks])

        # Use argmax to get class labels (0, 1, 2)
        pred = np.argmax(trial_prediction, axis=1)

        # Show classification report
        print(classification_report(self.Y_vali, pred, digits=3))

# Usage:
evaluation_metric = EvaluationMetric(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    Y_vali
)


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from transformers import TFRobertaModel

roberta_model = TFRobertaModel.from_pretrained("roberta-base", from_pt=True)
# "from_pt=True" converts a PyTorch model to TensorFlow.


In [ ]:
from tensorflow.keras import regularizers


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

with strategy.scope():
    # Define and compile your model
    model = Offense_classifier(input_shape=(100,), dropout_rate=0.3, dense_units=256)

    optimizer = Adam(learning_rate=3e-5)
    loss_fun = tf.keras.losses.SparseCategoricalCrossentropy()  # For integer labels 0,1,2
    metrics = ['accuracy']

    model.compile(optimizer=optimizer, loss=loss_fun, metrics=metrics)


In [ ]:
# Count samples per class
class_counts = np.bincount(Y_train)
total = len(Y_train)
print(f'Examples:\n    Total: {total}\n    Per-class counts: {class_counts}\n')

# Calculate weights: inverse frequency or max ratio
maxi = max(class_counts)
class_weight = {i: (maxi / count if count > 0 else 1.0) for i, count in enumerate(class_counts)}

for k, v in class_weight.items():
    print(f'Weight for class {k}: {v:.2f}')


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Save best model based on validation accuracy
checkpoint = ModelCheckpoint(
    'best_model.h5',                     # File to save
    monitor='val_accuracy',             # Metric to monitor
    verbose=1,
    save_best_only=True,                # Only keep the best
    mode='max'                          # 'max' because higher accuracy is better
)


In [ ]:
history = model.fit(
    x=[train_encodings["input_ids"], train_encodings["attention_mask"]],
    y=Y_train,
    validation_data=([val_encodings["input_ids"], val_encodings["attention_mask"]], Y_val),
    callbacks=[evaluation_metric, checkpoint],
    batch_size=64,
    shuffle=True,
    epochs=6,
    class_weight=class_weight
)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()


In [ ]:
test_set_encodings = tokenizer(X_test_set, max_length=100, truncation=True, padding="max_length", return_tensors='tf')

In [ ]:
answer = model.predict([test_set_encodings["input_ids"], test_set_encodings["attention_mask"]])

In [ ]:
import numpy as np

pred = []
for i in range(len(X_test_set)):
    num = answer[i]
    predicted_class = np.argmax(num)  # Select class with highest probability
    pred.append(predicted_class)


In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y_test_set, predictions=pred)
print(con_mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_names = ["Hate Speech", "Offensive Language", "Neither"]

plt.figure(figsize=(8, 8))
sns.set(font_scale=1.5)

sns.heatmap(
    con_mat,
    annot=True,
    cmap=plt.cm.viridis,
    fmt='d',
    xticklabels=class_names,
    yticklabels=class_names,
    annot_kws={"size": 15}
)

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import f1_score, classification_report

In [ ]:
f1_score(Y_test_set, pred, average='macro')

In [ ]:
print(classification_report(Y_test_set, pred, target_names=["Hate Speech", "Offensive Language", "Neither"], digits=3))


In [ ]:
answer_train = model.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])

In [ ]:
answer_train

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# 3 neuron output
model.layers[-6].name

In [ ]:
# Print all layer names
for layer in model.layers:
    print(layer.name)


In [ ]:
from tensorflow.keras.models import Model

# Find the output tensor of the desired layer by name
intermediate_output = [layer.output for layer in model.layers if layer.name == 'dense_12'][0]

# Create a new model that ends at that intermediate layer
cluster_dense_3 = Model(inputs=model.inputs, outputs=intermediate_output)


In [ ]:
cluster_3 = cluster_dense_3.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])


In [ ]:
pred_train = []

for i in range(len(Y)):
    score = (
        float(answer_train[i][0]) if isinstance(answer_train[i], (list, np.ndarray))
        else float(answer_train[i])
    )

    if 0 < score <= 0.3:
        pred_train.append(0)  # Hate Speech
    elif 0.7 <= score < 1:
        pred_train.append(1)  # Offensive Language
    elif 0.3 < score < 0.7:
        pred_train.append(2)  # Neither
    else:
        pred_train.append(2)  # For exact 0 or 1 (edge cases)

print("Sample of predicted labels:", pred_train[:10])


In [ ]:
import numpy as np

flag = []
x_ = []
y_ = []
z_ = []

for i in range(len(Y)):
    x_.append(cluster_3[i][1])
    y_.append(cluster_3[i][0])
    z_.append(cluster_3[i][2])

    score_raw = answer_train[i]

    # Extract first float if it's an array or list
    score = (
        float(score_raw[0])
        if isinstance(score_raw, (list, np.ndarray))
        else float(score_raw)
    )

    # Class assignment based on thresholds
    if 0 < score <= 0.3:
        flag.append(0)  # Hate Speech
    elif 0.7 <= score < 1:
        flag.append(1)  # Offensive Language
    elif 0.3 < score < 0.7:
        flag.append(2)  # Neither
    else:
        flag.append(2)  # For exact 0 or 1

from collections import Counter
print("Class distribution:", Counter(flag))


In [ ]:
Counter(flag)

In [ ]:
con_mat = tf.math.confusion_matrix(labels=Y, predictions=pred_train)
print(con_mat)

In [ ]:
import pandas as pd
import plotly.express as px

# Map class indices to new labels
pred_colour = []
for i in range(len(flag)):
    if flag[i] == 0:
        pred_colour.append("Hate Speech")
    elif flag[i] == 1:
        pred_colour.append("Offensive Language")
    elif flag[i] == 2:
        pred_colour.append("Neither")
    else:
        pred_colour.append("Unknown")  # Optional fallback

# Create DataFrame for Plotly
test_df = pd.DataFrame({'x': x_, 'y': y_, 'z': z_, 'Labels': pred_colour})

# Create 3D scatter plot
fig = px.scatter_3d(test_df, x='x', y='y', z='z', color='Labels')

# Style settings
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 0.7,
        'colorscale': 'Oryel',
    }
)
fig.update_layout(
    legend={'itemsizing': 'constant', 'font_size': 18},
    font_size=15,
    scene_aspectmode='cube',
    width=750,
    height=500,
    margin=dict(l=0, r=0, b=0, t=0)
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Map numerical predictions to labels
pred_colour = []
for i in range(len(pred_train)):
    if pred_train[i] == 0:
        pred_colour.append("Hate Speech")
    elif pred_train[i] == 1:
        pred_colour.append("Offensive Language")
    elif pred_train[i] == 2:
        pred_colour.append("Neither")
    else:
        pred_colour.append("Unknown")

# Create DataFrame for 3D visualization
test_df = pd.DataFrame({'X': x_, 'Y': y_, 'Z': z_, 'Labels': pred_colour})

# Generate the 3D scatter plot
fig = px.scatter_3d(test_df, x='X', y='Y', z='Z', color='Labels')

# Customize marker and layout
fig.update_traces(
    marker={
        'size': 1,
        'opacity': 1,
        'colorscale': 'rainbow',
    }
)
fig.update_layout(
    legend={'itemsizing': 'constant'},
    font_size=14,
    scene_aspectmode='cube',
    width=850,
    height=750,
    margin=dict(l=0, r=0, b=0, t=0)
)

fig.show()


In [ ]:
model.layers[-8].name

In [ ]:
from tensorflow.keras.models import Model

# Safely get the output of the layer using index (-8) or name if you know it
intermediate_output = model.layers[-8].output

# Create the intermediate model
cluster_bert = Model(inputs=model.inputs, outputs=intermediate_output)

# Predict using the strategy scope
with strategy.scope():
    cl_bert = cluster_bert.predict([cluster_encodings["input_ids"], cluster_encodings["attention_mask"]])


In [ ]:
len(cl_bert)

In [ ]:
flag_bert = []
x_bert = []
y_bert = []
z_bert = []

for i in range(len(Y)):
    # Append cluster coordinates (assuming cl_bert[i] = [x, y, z])
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    # Safely extract scalar score (handles if answer_train[i] is array-like)
    score = (
        float(answer_train[i][0]) if isinstance(answer_train[i], (list, np.ndarray))
        else float(answer_train[i])
    )

    # Assign class based on thresholds
    if 0 < score <= 0.28:
        flag_bert.append(0)   # Hate Speech
    elif 0.28 < score < 0.8:
        flag_bert.append(2)   # Neither
    elif 0.8 <= score < 1:
        flag_bert.append(1)   # Offensive Language
    else:
        flag_bert.append(2)   # fallback to Neither (for edge cases 0 or 1)

print(f"Total points processed: {len(Y)}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Convert coordinates into feature matrix (NumPy array)
X = np.array(list(zip(x_bert, y_bert, z_bert)))

# Convert labels to NumPy array
y = np.array(flag_bert)

# Split the data (stratify to maintain class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Initialize Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy of the Random Forest model: {accuracy:.4f}')

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
from sklearn.cluster import KMeans

# Ensure cl_bert is a NumPy array
import numpy as np
cl_bert_np = np.array(cl_bert)

# Fit KMeans with 3 clusters and random state 44
kmeans_bert = KMeans(n_clusters=3, random_state=44)
kmeans_bert.fit(cl_bert_np)

# Predict cluster labels for cl_bert data
y_kmeans_bert = kmeans_bert.predict(cl_bert_np)


In [ ]:
# 1 index values are sexist
# 0 index values are not sexist
# 2 index values are neutral

# Count how many times flag_bert == 2 (neutral) and predicted cluster is 1
count = 0
for i in range(len(flag_bert)):
    if flag_bert[i] == 2 and y_kmeans_bert[i] == 1:
        count += 1
print("Count of neutral samples predicted as cluster 1:", count)

# Swap cluster labels 0 and 1 in y_kmeans_bert, keep 2 as is
for i in range(len(y_kmeans_bert)):
    if y_kmeans_bert[i] == 0:
        y_kmeans_bert[i] = 1
    elif y_kmeans_bert[i] == 1:
        y_kmeans_bert[i] = 0
    # If it's 2, keep as 2 (neutral)

# Recompute flag_bert based on answer_train thresholds and coordinates from cl_bert
flag_bert = []
x_bert = []
y_bert = []
z_bert = []

for i in range(len(Y)):
    x_bert.append(cl_bert[i][0])
    y_bert.append(cl_bert[i][1])
    z_bert.append(cl_bert[i][2])

    score = answer_train[i]

    # Check if score is array-like and has more than 1 element
    if hasattr(score, 'shape') and len(score) > 1:
        # Example: take first element or the max
        score = score[0]   # or score = np.max(score) or np.argmax(score)
    else:
        # If it's a scalar-like array of size 1, get the scalar
        if hasattr(score, 'shape') and len(score) == 1:
            score = score.item()

    if 0 < score <= 0.3:
        flag_bert.append(0)  # Not sexist
    elif 0.7 <= score < 1:
        flag_bert.append(1)  # Sexist
    elif 0.3 < score < 0.7:
        flag_bert.append(2)  # Neutral
    else:
        flag_bert.append(2)  # Default neutral

print("Total samples processed:", len(flag_bert))



# Compute confusion matrix between true labels and predicted clusters
import tensorflow as tf
con_mat = tf.math.confusion_matrix(labels=flag_bert, predictions=y_kmeans_bert)
print("Confusion matrix:")
print(con_mat.numpy())


In [ ]:
import sklearn
print(sklearn.metrics.classification_report(flag_bert, y_kmeans_bert, output_dict=False, digits=3))

In [ ]:
from scipy.stats import mode
import numpy as np

def match_clusters(true_labels, cluster_labels):
    labels = np.unique(true_labels)
    clusters = np.unique(cluster_labels)
    label_map = {}

    for cluster in clusters:
        mask = cluster_labels == cluster
        cluster_true_labels = true_labels[mask]

        if len(cluster_true_labels) == 0:
            # No samples in this cluster, assign a default label, say -1
            label_map[cluster] = -1
            continue

        mode_result = mode(cluster_true_labels)

        # Check if mode_result.mode is scalar or array
        if hasattr(mode_result.mode, '__len__'):
            most_common_label = mode_result.mode[0]
        else:
            most_common_label = mode_result.mode

        label_map[cluster] = most_common_label

    new_preds = np.array([label_map.get(cl, -1) for cl in cluster_labels])
    return new_preds

flag_bert_np = np.array(flag_bert)
y_kmeans_bert_np = np.array(y_kmeans_bert)

mapped_preds = match_clusters(flag_bert_np, y_kmeans_bert_np)

from sklearn.metrics import classification_report
print(classification_report(flag_bert_np, mapped_preds, digits=3))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [ ]:
centers_bert = kmeans_bert.cluster_centers_

In [ ]:
from scipy.spatial.distance import cosine

# Assuming 3 clusters (offensive, non-offensive, neutral)
cluster_labels = ['offensive', 'non_offensive', 'neutral']

# Initialize dictionary to hold similarity scores for each cluster
svns = {label: [] for label in cluster_labels}

for i in range(len(Y)):
    sample = cl_bert[i]
    for idx, label in enumerate(cluster_labels):
        # cosine distance divided by 2, then converted to similarity (1 - value)
        sim = 1 - (cosine(sample, centers_bert[idx]) / 2)
        svns[label].append(sim)

# Example: print lengths to verify all lists are complete
for label in cluster_labels:
    print(f"Length of svns_{label}: {len(svns[label])}")

# Access individual lists:
svns_off = svns['offensive']
svns_noff = svns['non_offensive']
svns_neu = svns['neutral']


In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
pred_colour = []
for i in range(len(Y)):
    if y_kmeans_bert[i] == 2:
        pred_colour.append("Neutral")
    elif y_kmeans_bert[i] == 1:
        pred_colour.append("Not Offensive")
    elif y_kmeans_bert[i] == 0:
        pred_colour.append("Offensive")
    else:
        pred_colour.append("Unknown")  # fallback just in case

# Prepare dataframe for plotting
test_df = pd.DataFrame({
    'SVNS_Offensive': svns_off,
    'SVNS_Not_Offensive': svns_noff,
    'SVNS_Neutral': svns_neu,
    'Labels': pred_colour
})

fig = px.scatter_3d(
    test_df,
    x='SVNS_Offensive',
    y='SVNS_Not_Offensive',
    z='SVNS_Neutral',
    color='Labels',
    color_discrete_map={
        "Offensive": "red",
        "Not Offensive": "green",
        "Neutral": "blue",
        "Unknown": "gray"
    },
    title="3D Scatter of SVNS Scores with Cluster Labels"
)

fig.update_traces(marker=dict(size=3, opacity=0.8))
fig.update_layout(
    legend=dict(itemsizing='constant', font_size=16),
    font_size=14,
    scene_aspectmode='cube',
    width=900,
    height=700,
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()


In [ ]:
num=len(Y)

In [ ]:
from sklearn.metrics import classification_report

pred_krobert = []

for i in range(num):
    # Check which SVNS score is the highest for each sample
    if svns_off[i] > svns_neu[i] + svns_noff[i]:
        pred_krobert.append(0)  # Offensive
    elif svns_noff[i] > svns_neu[i] and svns_noff[i] > svns_off[i]:
        pred_krobert.append(1)  # Not Offensive
    elif svns_neu[i] > svns_noff[i] and svns_neu[i] > svns_off[i]:
        pred_krobert.append(2)  # Neutral
    else:
        pred_krobert.append(0)  # Default/fallback to Offensive

print(classification_report(Y, pred_krobert, output_dict=False, digits=3))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# Prepare feature matrix and labels
X = np.column_stack((svns_off, svns_noff, svns_neu))  # Shape: (num_samples, 3)
y = np.array(Y)  # True labels

# Split into train and test sets (e.g., 70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Initialize and train logistic regression model with balanced class weights to handle imbalance
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate model
print(classification_report(y_test, y_pred, digits=3))


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# Stack the SVNS features as input vectors
X = np.column_stack((svns_off, svns_noff, svns_neu))  # Shape (num_samples, 3)

# True labels array (0 = Hate Speech, 1 = Offensive, 2 = Normal)
y = np.array(Y)

# Split into train and test sets, stratified to keep class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Initialize Logistic Regression with balanced class weights (important if classes are imbalanced)
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42, multi_class='ovr')

# Train the model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Show detailed classification report
print(classification_report(y_test, y_pred, digits=3))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np

X = np.column_stack((svns_off, svns_noff, svns_neu))
y = np.array(Y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

rf_model = RandomForestClassifier(
    n_estimators=200, random_state=42, class_weight='balanced'
)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

print(classification_report(y_test, y_pred, digits=3))
